## Chroma & Retriever

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install chromadb tiktoken transformers sentence_transformers openai langchain tablib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.1 MB/s eta 0:00:00


In [5]:
import os
import openai
os.environ["OPENAI_API_KEY"] = ""

In [6]:
# 필요한 라이브러리 임포트
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import CSVLoader
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
import numpy as np

In [7]:
# 코사인 유사도 함수 정의
def cos_sim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

In [8]:
# 토크나이저 설정
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

## 한국어 사전 학습 모델임베딩 -ko-sbert-nli

In [8]:
# 오픈 소스 임베딩 함수 생성
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': 'True'}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.46k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Faiss

In [16]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 50.0 MB/s eta 0:00:00


In [19]:
from langchain.text_splitter import CharacterTextSplitter

with open('/content/drive/MyDrive/filtered_df.csv') as f:
  state_of_the_union = f.read()

# 텍스트 분할기 초기화
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 100,
    chunk_overlap = 10,
    length_function = len,
)

texts = text_splitter.split_text(state_of_the_union)
texts = texts[1:]

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.


In [20]:
char_list = []
for i in range(len(texts)):
  char_list.append(len(texts[i]))

In [32]:
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

class TextEmbeddingFAISS:
    def __init__(self, model_name="jhgan/ko-sbert-nli"):
        self.model = SentenceTransformer(model_name)
        self.index = None

    def get_sentence_embeddings(self, sentences):
        # 문장을 임베딩 벡터로 변환
        embeddings = self.model.encode(sentences)
        return embeddings

    def create_faiss_index_and_search(self, embeddings, k):
        # FAISS 인덱스 생성 및 벡터 추가
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(embeddings)

        # 유사 벡터 검색
        nearest_neighbors = []
        for i in range(embeddings.shape[0]):
            D, I = self.index.search(np.expand_dims(embeddings[i], axis=0), k + 1)
            neighbors = I[0][1:k+1]  # 자기 자신 제외
            distances = D[0][1:k+1]
            nearest_neighbors.append((neighbors, distances))

        return nearest_neighbors

# 데이터 적용
sentences = texts

# 클래스 인스턴스 생성 및 사용 예제
embedding_faiss = TextEmbeddingFAISS()
embeddings = embedding_faiss.get_sentence_embeddings(sentences)
nearest_neighbors = embedding_faiss.create_faiss_index_and_search(embeddings, k=2)

# 최근접 이웃 출력
for i, (neighbors, distances) in enumerate(nearest_neighbors):
    print(f"문장 {i}의 최근접 이웃:", neighbors)
    print(f"문장 {i}의 거리:", distances)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
문장 11329의 최근접 이웃: [13541 11138]
문장 11329의 거리: [1.8580892e-10 1.4781688e+02]
문장 11330의 최근접 이웃: [13542   766]
문장 11330의 거리: [  0.      136.59729]
문장 11331의 최근접 이웃: [13543 13547]
문장 11331의 거리: [  0.      133.73172]
문장 11332의 최근접 이웃: [13544 11141]
문장 11332의 거리: [  0.      137.02435]
문장 11333의 최근접 이웃: [13545 11139]
문장 11333의 거리: [  0.      135.54153]
문장 11334의 최근접 이웃: [13546  4659]
문장 11334의 거리: [6.0014702e-11 1.1438063e+02]
문장 11335의 최근접 이웃: [13547  4410]
문장 11335의 거리: [7.6099682e-11 1.2953673e+02]
문장 11336의 최근접 이웃: [13548 13602]
문장 11336의 거리: [ 0.      88.12565]
문장 11337의 최근접 이웃: [13549 11343]
문장 11337의 거리: [6.1707320e-11 1.1014764e+02]
문장 11338의 최근접 이웃: [13550  4092]
문장 11338의 거리: [  0.      113.46484]
문장 11339의 최근접 이웃: [13551 11345]
문장 11339의 거리: [  0.      107.04265]
문장 11340의 최근접 이웃: [13552  3464]
문장 11340의 거리: [  0.      132.33958]
문장 11341의 최근접 이웃: [13553 11337]
문장 11341의 거리: [  0.      117.08948]
문장 11342의 최근접 이웃: [13554  7058]
문장 11342의 거리: [  0

## Chroma

In [9]:
import csv
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document  # Document 클래스 임포트

In [10]:
# CSV 파일 로드 및 문장 텍스트 추출
csv_path = "/content/drive/MyDrive/filtered_df.csv"  # 여기에 CSV 파일 경로를 넣기
sentence_texts = []
with open(csv_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        sentence_texts.append(row[1])  # CSV 파일의 각 행의 두 번째 열에 있는 문장 내용을 리스트에 추가

In [11]:
# HuggingFaceEmbeddings 인스턴스 생성
hf = HuggingFaceEmbeddings()

# 문장 텍스트를 이용하여 임베딩 생성
sentence_embeddings = hf.embed_documents(sentence_texts)

# Document 객체 생성
documents = [Document(page_content=text) for text in sentence_texts]

# Chroma 데이터베이스에 문서 저장
db = Chroma.from_documents(documents, hf, persist_directory="./chroma_db")

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets. (raised from /usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88)


In [12]:
# 사용자 입력을 받아서 임베딩 생성
user_input = input("질문을 입력하세요: ")  # 실제로는 동적으로 받아야 함

# 문장 텍스트를 이용하여 임베딩 생성
query_embedding = hf.embed_documents([user_input])[0]

# Chroma 데이터베이스 불러오기
db3 = Chroma(persist_directory="./chroma_db", embedding_function=hf)


질문을 입력하세요: 나 남자친구랑 시간 약속 문제 때문에 싸웠어.


In [13]:
# 유사도 계산 및 유사한 문장 검색
docs_with_scores = db3.similarity_search(user_input, k=1)

In [14]:
# 결과 반환 함수
def retrieve_most_similar_sentence(docs_with_scores):
    if docs_with_scores:
        most_similar_document = docs_with_scores[0]  # 첫 번째 결과의 문서 객체
        return most_similar_document.page_content  # 문장 내용 반환
    else:
        return None

In [15]:
# 리트리버에게 반환할 값
most_similar_sentence = retrieve_most_similar_sentence(docs_with_scores)
print("가장 유사한 문장:", most_similar_sentence)

가장 유사한 문장: 자기야 나 요즘 너무 감정이 불안하고 예민한 것 같아.


## Retriever

In [16]:
from langchain.retrievers import EnsembleRetriever
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.retrievers.bm25 import BM25Retriever
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

In [121]:
bm25_retriever = BM25Retriever.from_texts(
    sentences
)

In [122]:
print(bm25_retriever)

vectorizer=<rank_bm25.BM25Okapi object at 0x7c2796dcca60>


In [129]:
# 앙상블 Retriever

# Chroma 데이터베이스를 retriever에서 검색 가능한 형태로 변환
retriever = db.as_retriever()

# BM25 retriever, faiss retriever 초기화
bm25_retriever = BM25Retriever.from_texts(
    sentences
)
bm25_retriever.k=1

# 임베딩 기반 (문서 검색) 리트리버 생성
embedding = OpenAIEmbeddings()
faiss_vectorstore = FAISS.from_texts(
    sentences,
    embedding)

# faiss_retriever -> search_k = 최근접 이웃 수, k = k개의 관련성 높은 문서 반환
faiss_retriever = faiss_vectorstore.as_retriever(
    search_k=20,
    k=1
)

# Ensemble Retriever 생성
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever],
    search_type="mmr"
)

In [130]:
ensemble_result = ensemble_retriever.get_relevant_documents(user_input)
print(ensemble_result)

[Document(page_content='0,친구가 나를 대수롭지 않게 생각한다는 기분이 들었어요. 나와의 약속은 시간이 임박해서도 취소할 만큼 가볍게 여기는 게 너무 상처가 되더군요.,50,pain'), Document(page_content='0,갑자기 그 친구가 종종 약속을 지키지 않았던 사실이 떠올랐어요. 먼저 만나자고 하면서도 시간을 정하지 않거나 당일 취소한 적이 여러 번 있거든요. 그것들이 겹쳐져서 더 상처받았다고 느껴지나 봐요.,50,pain'), Document(page_content='1,자기 원래 약속 시간 칼같이 지키는 사람이잖아. 비양심적인 기사님 때문에 돈도 더 내야 되고 약속도 늦으면 나라도 정말 화날 거 같아.,3,anger')]
